In [384]:
#import Files 
import xml.sax
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
import bz2
from nltk.stem.porter import *
import sys
from nltk.stem.snowball import SnowballStemmer



[nltk_data] Downloading package stopwords to /home/apurvi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [385]:
#global Variables 
Doc_ID=1 #to maintain total Number of pages in the dump
Doc_IDtoTitle={} # a map to maintain name of doc ids 

In [386]:
class TitleProcessor():
    def processData(self,data):
#         data=self.camelCase(data)
#         print(data)
        data=self.tokenizer(data)
        data=self.case_fold(data)
        data=self.removeStopwords(data)
        data=self.doStemming(data)
        return data
        
#     def camelCase(self,data):
#         return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))',data)
    
    def case_fold(self,data):
        case_Folded= [w.casefold() for w in data]
        return case_Folded
    
    def tokenizer(self,data):
        tokenizedata=re.findall("\d+|[\w]+",data)
        tokenizedata=[key for key in tokenizedata]
        return tokenizedata
    
    def removeStopwords(self,data):
        stop_word = set(stopwords.words('english'))
        removedStopW=[w for w in data if w not in stop_word] 
        return removedStopW
    
    def doStemming(self,data):
        stemmer=SnowballStemmer("english")
        stemmed_data=[]
        for words in data:
            stemmed_data.append(stemmer.stem(words))
        return stemmed_data

In [387]:
class TextFieldProcessor():
    data=None
    def process(self,data):
        self.data=data
        self.data=self.data.casefold()
        self.remove_Regx()
        info=self.get_InfoBox_category()
        print("info",info)
        return self.data
    
    def remove_Regx(self):
        removReg1=re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL).sub('',self.data)
        self.data=removReg1
        
        removReg2=re.compile(r'{\|(.*?)\|}',re.DOTALL).sub('',self.data)
        self.data=removReg2
        
        removReg3=re.compile(r'{{v?cite(.*?)}}',re.DOTALL).sub('',self.data)
        self.data=removReg3
        
        removReg4=re.compile(r'[.,;_()"/\']',re.DOTALL).sub(' ',self.data)
        self.data=removReg4
        
        removReg5=re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL).sub('',self.data)
        self.data=removReg5
        
        removReg6=re.compile(r'<(.*?)>',re.DOTALL).sub('',self.data)
        self.data=removReg6
        self.data=self.data.replace('_',' ').replace(',','')
        
    def get_InfoBox(self):
        info=[]
        flag=0
        lines=self.data.split('\n')
        no_line=len(lines)
        i=0
        while(i<no_line):
            if '{{infobox' in lines[i]:
                count_open=0
                split_1=lines[i].split('{{infobox')
                if(len(split_1)==2):
                    info.extend(split_1[1:])
                while True:
                    if '{{' in lines[i]:
                        openb=lines[i].count('{{')
                        flag=1
                        count_open+=openb
                    
                    if '}}' in lines[i]:
                        close=lines[i].count('}}')
                        flag=0
                        count_open-=close
                    
                    if count_open<=0:
                        break
                    i=i+1
                    append_data=lines[i]
                    if(i<len(lines)):
                        info.append(append_data)
                    else:
                        break
            elif 
            i+=1
        return info
        

In [388]:
def check():
    global Doc_IDtoTitle
    for key in Doc_IDtoTitle:
        print(key," ",Doc_IDtoTitle[key])

In [389]:
#content Handler code for XML parser
def storeValue(key,value,Dict):
    global Doc_IDtoTitle
    Doc_IDtoTitle[key]=value
    
class parseXML(xml.sax.ContentHandler):
    
    global Doc_ID,Doc_IDtoTitle 
    def __init__(self):
        xml.sax.ContentHandler.__init__(self)
        self.current_tag=""
        self.title_content=""
        self.text_content=""
    
    def startElement(self,tag,attributes):
        self.current_tag=tag
        if tag=='page':
            self.title_content=""
            self.text_content=""

    def characters(self,content):
        if self.current_tag=="title":
            self.title_content+=content
        elif self.current_tag=="text":
            self.text_content+=content
            
    def endElement(self,tag):
        if tag=='page':
            global Doc_ID
            print("----------------------end doc",Doc_ID,"----------------------------")
            if Doc_ID==100:
#                 check()
                sys.exit()
            Doc_ID+=1
        
        elif tag=='title':
#             print(self.title_content)
            storeValue(Doc_ID,self.title_content,Doc_IDtoTitle) 
        
        elif tag=='text':
            #process title
            pr=TitleProcessor()
            processedTitle=pr.processData(self.title_content)
            #print("Processed title : ",processedTitle)
            
            #get fields from text_content
            pt=TextFieldProcessor()
            pt.process(self.text_content)
        self.current_tag=""

In [390]:
if __name__=="__main__":
    Wiki_input_file="./Data/input.bz2"
    source = bz2.BZ2File(Wiki_input_file, "rb")

    #Create a parser Object
    parser= xml.sax.make_parser()

    #turnoff namespaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    
    #callHandler
    handler= parseXML()
    
    #parse data 
    parser.setContentHandler(handler)
    parser.parse(source)
    

info []
----------------------end doc 1 ----------------------------
info []
----------------------end doc 2 ----------------------------
info []
----------------------end doc 3 ----------------------------
info []
----------------------end doc 4 ----------------------------
info []
----------------------end doc 5 ----------------------------
info []
----------------------end doc 6 ----------------------------
info []
----------------------end doc 7 ----------------------------
info []
----------------------end doc 8 ----------------------------
info []
----------------------end doc 9 ----------------------------
info []
----------------------end doc 10 ----------------------------
info []
----------------------end doc 11 ----------------------------
info [' medical condition  new ', '| name          = autism', '| image         = autism-stacking-cans 2nd edit jpg', '| alt           = boy stacking cans', '| caption       = repetitively stacking or lining up objects is commonly associate

info [' officeholder', '| name = abraham lincoln', '| image = abraham lincoln o-77 matte collodion print jpg', '| caption = lincoln in november 1863', '| alt = an iconic photograph of a bearded abraham lincoln showing his head and shoulders ', '| office = 16th [[president of the united states]]', '| vicepresident = [[hannibal hamlin]]  1861–65  andrew johnson  mar –apr  1865  ', '| term start = march 4  1861', '| term end = april 15  1865', '| predecessor = [[james buchanan]]', '| successor = [[andrew johnson]]', '| state1 = [[illinois]]', '| district1 = {{ushr|il|7|7th}}', '| term start1 = march 4  1847', '| term end1 = march 3  1849', '| predecessor1 = [[john henry  representative |john henry]]', '| successor1 = [[thomas l  harris]]', '| office2 = member of the[[illinois house of representatives]]from [[sangamon county]]', '| term start2 = december 1  1834', '| term end2 = december 4  1842', '| birth date = {{birth date|1809|2|12}}', '| birth place = [[sinking spring farm]]  [[kentuc

info [' writer', '| name = ayn rand', '| native name = алиса зиновьевна розенбаум{{nowrap| alisa zinovyevna rosenbaum }}', '| image = ayn rand  1943 talbot portrait  jpg', '| alt = black and white photo of a white woman facing the camera with her body turned to the side ', '| caption = rand in 1943', '| birth date = {{birth date|mf=yes|1905|02|02}}', '| birth place = [[saint petersburg|st  petersburg]]  [[russian empire]]', '| death date = {{death date and age|mf=yes|1982|3|6|1905|2|2}}', '| death place = [[manhattan]]  [[new york city]]  [[united states|u s ]]', '| resting place = [[kensico cemetery]]  [[valhalla  new york]]  united states', '| pseudonym = ayn rand', '| occupation = writer', '| language = english', '| citizenship = {{plainlist|', '* [[russian empire]]  1905–1917  ', '* [[russian republic]]  1917 ', '* [[russian sfsr]]  1917–1922 ', '* [[soviet union]]  1922–1931 {{efn|left the country in 1926 }}', '* [[united states]]  1931–1982 ', '}}', '| alma mater = [[saint peters

info []
----------------------end doc 89 ----------------------------
info []
----------------------end doc 90 ----------------------------
info []
----------------------end doc 91 ----------------------------
info []
----------------------end doc 92 ----------------------------
info []
----------------------end doc 93 ----------------------------
info []
----------------------end doc 94 ----------------------------
info [' character encoding|', '| name = ascii', '| alias = iso-ir-006{{citation |mode=cs1 |url= |title=iso-ir-006: ascii graphic character set |author=ansi |author-link=american national standards institute |publisher=itscj [[information processing society of japan|ipsj]] |date=1975-12-01}}  ansi x3 4-1968  ansi x3 4-1986  iso 646 irv:1991  iso646-us  us  ibm367  cp367', '| mime = us-ascii', '| image = usascii code chart png', '| caption = ascii chart from a pre-1972 printer manual', '| lang = [[english language|english]]', '| extensions = * [[unicode]]', '* [[iso iec 8859]

SystemExit: 